<a href="https://colab.research.google.com/github/EvgSkv/logica/blob/main/examples/Logica_example_News_connections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logica example: News connections



Here we find a sequence in the news that connects pop singer Justin Bieber to the Lieutenant Governor of British Columbia Janet Austin.

We use [GDELT](https://www.gdeltproject.org/data.html) public BigQuery table.

## Install Logica

In [ ]:
!pip install logica

## Extract the news graph

In [ ]:
!mkdir lib

mkdir: cannot create directory ‘lib’: File exists


In [ ]:
%%writefile lib/util.l
    
ExtractPeople(people_str) = people :-
  people List= (
      person :-
      person in Split(people_str, ";"),
      # Mistakes in the dataset:
      person != "los angeles",
      person != "las vegas"
  );

Gdelt2020(..r) :-
  `gdelt-bq.gdeltv2.gkg`(..r),
  Substr(ToString(r.date), 0, 4) == "2020";


Overwriting lib/util.l


In [ ]:
from logica import colab_logica

In [ ]:
%%logica SaveEdges

import lib.util.ExtractPeople;
import lib.util.Gdelt2020;

@Dataset("logica_graph_example");

EdgeWeight(person1, person2, url? AnyValue= url) += 1 :-
  Gdelt2020(persons:, documentidentifier: url),
  people == ExtractPeople(persons),
  person1 in people,
  person2 in people,
  person1 != person2;

@Ground(Edge);
Edge(a, b, url:) :- EdgeWeight(a, b, url:) > 120;

SaveEdges() += 1 :- Edge();

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

The following table is stored at SaveEdges variable.


logica_value
0       1490748

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Finding all paths from Justin Bieber

In [ ]:
%%logica SaveP3

@Dataset("logica_graph_example");
@Ground(Edge);

Root() = "justin bieber";

P0(person: Root(), path: ["__start__", Root()]);

@Ground(P1);
P1(person:, path? AnyValue= path) distinct :-
  P0(person: prev_person, path: prev_path),
  Edge(prev_person, person),
  ~P0(person:),
  path == ArrayConcat(prev_path, [person]);
P1(person:, path? AnyValue= path) distinct :-
  P0(person:, path:);

# Few steps of recursion are convenient to describe via
# repeated functor calls.
P2 := P1(P0: P1);
P3 := P2(P1: P2); 

SaveP3() += 1 :- P3();



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

The following table is stored at SaveP3 variable.


logica_value
0         64296

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Printing the path

In [ ]:
%%logica PathExplanation

@Dataset("logica_graph_example");
@Ground(P3);
@Ground(Edge);

@Ground(ThePath);
ThePath() = path :-
  P3(person: "janet austin", path:);


@OrderBy(PathExplanation, "step");
PathExplanation(step: i, from_person: p1, to_person: p2, url:) :-
  path == ThePath(),
  i in RangeOf(path),
  i > 0,
  i < ArrayLength(path) - 1,
  p1 == Element(path, i),
  p2 == Element(path, i + 1),
  Edge(p1, p2, url:);


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

The following table is stored at PathExplanation variable.


step  ...                                                url
0     1  ...  https://misionesonline.net/2020/06/28/espectac...
1     2  ...  https://www.macleans.ca/politics/ottawa/john-h...

[2 rows x 4 columns]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
for step, from_person, to_person, url in zip(PathExplanation['step'],
                                             PathExplanation['from_person'],
                                             PathExplanation['to_person'],
                                             PathExplanation['url']):
  print("%s is (or was) connected to %s by URL: %s" % (from_person, to_person, url))
  

justin bieber is (or was) connected to justin trudeau by URL: https://misionesonline.net/2020/06/28/espectaculos-el-concierto-virtual-global-goal-unite-for-our-future-junto-donaciones-para-la-lucha-contra-el-coronavirus/
justin trudeau is (or was) connected to janet austin by URL: https://www.macleans.ca/politics/ottawa/john-horgan-savvy-opportunist-or-practical-realist/
